In [19]:
import numpy as np

In [20]:
class KMeans:
    def __init__(self, k, max_iterations=10000):
        self.k = k
        self.max_iterations = max_iterations
        self.centroids = None
        self.labels = None

    def initializeCentroids(self, data):
        # Inicialización de centroides de manera aleatoria
        np.random.seed(0)
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        centroids = data[indices]
        return centroids
    
    def initializeCentroidsFeatures(self, data, real_labels):
        vectoreded_images = data.reshape(data.shape[0], -1)  # Aplanar las imágenes en vectores de características
        UniqueLabels = np.unique(real_labels)

        # Cálculo de características promedio por etiqueta
        centroids = []
        for label in UniqueLabels:
            label_images = vectoreded_images[real_labels == label]
            label_mean = np.mean(label_images, axis=0)
            centroids.append(label_mean)

        # Selección de los primeros K centroides iniciales
        centroids = np.array(centroids)[:self.k]
        return centroids

    def AssignClusters(self, data):
        distances = np.sqrt(((data[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        return labels

    def UpdateCentroids(self, data):
        # Actualización de los centroides como la media de los puntos asignados a cada clúster
        centroids = np.zeros((self.k, data.shape[1]))
        for i in range(self.k):
            cluster_data = data[self.labels == i]
            if len(cluster_data) > 0:
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fitRANDOM(self, data):
        self.centroids = self.initializeCentroids(data)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.AssignClusters(data)
            self.centroids = self.UpdateCentroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break
            
    def fitPreprocess(self, data, real_labels):
        self.centroids = self.initializeCentroidsFeatures(data, real_labels)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.AssignClusters(data)
            self.centroids = self.UpdateCentroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break

    def predict(self, data):
        labels = self.AssignClusters(data)
        return labels
    
    

In [21]:
def ConfussionMatrix(true_labels, predicted_labels, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes))
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        confusion_matrix[true_label][predicted_label] += 1
    return confusion_matrix

In [22]:
def compareKmeans(X_train ,Y_train, X_test, Y_test, k = 10 ):
    kmeansPre = KMeans(k)
    kmeansR = KMeans(k)
    
    # Ajustar el modelo a los datos
    kmeansPre.fitPreprocess(X_train, Y_train)
    kmeansR.fitRANDOM(X_train)
    
    # Obtener las etiquetas asignadas a cada punto
    predicted_labelsPre = kmeansPre.predict(X_test)
    predicted_labelsR = kmeansR.predict(X_test)
    
    # Calcular la matriz de confusión
    confusion_matrixPre = ConfussionMatrix(Y_test, predicted_labelsPre, num_classes=k)
    confusion_matrixR = ConfussionMatrix(Y_test, predicted_labelsR, num_classes=k)
   
    # Imprimir la matriz de confusión
    '''print(confusion_matrixPre)
    print(confusion_matrixR)'''

    # Calcular la precisión como la suma de las coincidencias diagonales dividida por el número total de puntos
    accuracyPre = np.sum(np.max(confusion_matrixPre, axis=1)) / len(X_test)
    accuracyR = np.sum(np.max(confusion_matrixR, axis=1)) / len(X_test)

    # Imprimir la precisión
    print("Precisión de clasificación con preprocesamiento: {:.2f}%".format(accuracyPre * 100))
    print("Precisión de clasificación con Random: {:.2f}%".format(accuracyR * 100))

In [23]:
def readArchive(fileName):
    with open(fileName, 'r') as file:
        lines = file.readlines()

    # Elimina los caracteres de salto de línea y divide los valores por comas
    data = [line.strip().split(',') for line in lines]

    # Convierte la lista en una matriz de numpy
    return np.array(data, dtype=int)

dataTrain = readArchive('optdigits.tra')

X_train = dataTrain[:, :64]
Y_train = dataTrain[:, 64]

dataTest = readArchive('optdigits.tes')
X_test = dataTest[:, :64]
Y_test = dataTest[: , 64]

In [24]:
compareKmeans(X_train,Y_train,X_test, Y_test)

Precisión de clasificación con preprocesamiento: 86.31%
Precisión de clasificación con Random: 75.79%
